In [1]:
import os
import sys
import joblib
import FE_0823 as FE
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
import imp
imp.reload(FE)

<module 'FE_0823' from '/Users/jinseok/Jupyter/bigcon/Shopping_GG/2. FE/FE_0823.py'>

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
sale = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'sale_data_v05_0828.xlsx'))
meta = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'sale_meta_v04_0823.xlsx'))
item = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'item_meta_v04_0828.xlsx'))

In [9]:
sale['방송년도'] = sale['방송일시'].dt.year

In [10]:
sale.loc[sale['방송년도'] == 2020]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,결제방법,옵션,종류,브랜드,상품명다시,단위,방송년도
38300,2020-01-01 00:00:00,20.000,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.000,818.000,쿠첸 압력밥솥 10인용,2.000,NaN,NaN,쿠첸,압력밥솥,10인용,2020
38301,2020-01-01 00:00:00,nan,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.000,817.000,쿠첸 압력밥솥 6인용,2.000,NaN,NaN,쿠첸,압력밥솥,6인용,2020
38302,2020-01-01 00:00:00,nan,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.000,818.000,쿠첸 압력밥솥 10인용,1.000,NaN,NaN,쿠첸,압력밥솥,10인용,2020
38303,2020-01-01 00:00:00,nan,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,46608000.000,817.000,쿠첸 압력밥솥 6인용,1.000,NaN,NaN,쿠첸,압력밥솥,6인용,2020
38304,2020-01-01 00:20:00,20.000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,2020
38305,2020-01-01 00:40:00,20.000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,2020
38306,2020-01-01 01:00:00,20.000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,2020
38307,2020-01-01 01:20:00,20.000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,2020
38308,2020-01-01 01:40:00,17.167,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,2020


In [5]:
sale.shape, meta.shape, item.shape

((38309, 16), (2042, 15), (1364, 15))

# 01. Feature Engineering

In [6]:
sale = FE.engineering_TimeDiff(sale)

In [7]:
sale = FE.engineering_Soldout(sale)

In [8]:
sale = FE.engineering_DatePrice(sale)

연도입력


 2019


In [9]:
sale = FE.engineering_trend(sale)

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/polynomial.py:630: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale


# 02. 외부데이터 불러오기

In [10]:
# 경제지표 가져오기(월별)

In [11]:
df_eco = pd.read_excel(os.path.join('..', '..', '0.Data', '03_외부데이터', '전처리', 'prep_1차외부데이터_0901.xlsx'))

In [12]:
# 날씨 데이터 가져오기(시간별)

In [13]:
df_wth = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '전처리', 'prep_2019_pb_weather.csv'), encoding = 'cp949')

In [14]:
# 미세먼지 데이터 가져오기

In [15]:
df_dust = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '전처리', 'prep_2019_pb_dust.csv'), encoding = 'cp949')

# 03. 임베딩 데이터 불러오기

In [16]:
emb = pd.read_excel(os.path.join('..', '..', '0.Data', '04_임베딩데이터', 'version3_dim30.xlsx'), index_col = 0)

In [17]:
sale = sale.merge(emb.drop_duplicates(), on = 'NEW상품명', how = 'left')

In [18]:
sale.shape

(38309, 92)

# 04. 데이터 합치기

In [19]:
data = sale.merge(df_eco[df_eco['년도'] == 2019], left_on = '방송월', right_on = '월', how = 'left').drop(['월'], axis = 1)

In [20]:
data['방송일'] = data['방송일시'].dt.day

In [21]:
data = data.merge(df_wth, left_on = ['방송월', '방송일', '방송시간(시간)'], right_on = ['월', '일', '시간'], how ='left').drop(['월', '일', '시간'], axis = 1)

In [22]:
data = data.merge(df_dust, left_on = ['방송월', '방송일', '방송시간(시간)'], right_on = ['월', '일', '시간'], how ='left').drop(['월', '일', '시간'], axis = 1)

In [23]:
# 444개로 늘어난 featurea
data.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,NEW상품코드,NEW상품명,...,평균PM10_울산,평균PM10_인천,평균PM25_경기,평균PM25_광주,평균PM25_대구,평균PM25_대전,평균PM25_부산,평균PM25_서울,평균PM25_울산,평균PM25_인천
0,2019-01-01 06:00:00,20.000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.000,865.000,테이트 남성 셀린니트 3종,...,28.471,39.000,31.146,22.750,21.867,39.818,22.792,29.000,18.824,25.625
1,2019-01-01 06:00:00,20.000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.000,866.000,테이트 여성 셀린니트 3종,...,28.471,39.000,31.146,22.750,21.867,39.818,22.792,29.000,18.824,25.625
2,2019-01-01 06:20:00,20.000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.000,865.000,테이트 남성 셀린니트 3종,...,28.471,39.000,31.146,22.750,21.867,39.818,22.792,29.000,18.824,25.625
3,2019-01-01 06:20:00,20.000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.000,866.000,테이트 여성 셀린니트 3종,...,28.471,39.000,31.146,22.750,21.867,39.818,22.792,29.000,18.824,25.625
4,2019-01-01 06:40:00,20.000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.000,865.000,테이트 남성 셀린니트 3종,...,28.471,39.000,31.146,22.750,21.867,39.818,22.792,29.000,18.824,25.625


# 05. feature 조정

In [24]:
categorys = ['결제방법', '상품군_가격대', '전체_가격대', '상품군', '방송시간(시간)', '성별']
drop_columns = ['방송일시','마더코드', '상품코드', '상품명', 'NEW상품코드', 'NEW상품명', '단위', '브랜드', '취급액', '상품코드', '옵션', '종류', '년도', '날짜_x', '날짜_y', '날짜', '상품명다시']

In [25]:
for c in categorys:
    data[c] = data[c].astype(str)

In [26]:
# 예측 상품 중 판매가 0인 프로그램 실적은 예측에서 제외함 -> 무형 제외
data = data.loc[data['상품군'] != '무형']

In [28]:
# 주문이 0인, 취급액이 0인 데이터 제외함
data = data.loc[data['취급액'].notnull()]

In [29]:
y = data['취급액']
drop_data = data[drop_columns]
data = data.drop(drop_columns, axis = 1)

In [30]:
data['log최근3개월상품군추세'] = data['log최근3개월상품군추세'].astype(float)

In [31]:
X = pd.get_dummies(data)

In [32]:
X.shape, y.shape

((35379, 481), (35379,))

In [33]:
import joblib

In [34]:
joblib.dump({
    'X' : X,
    'y' : y
}, os.path.join('..', '..', '0.Data', '05_분석데이터', 'Snd_FE.pkl'))

['../../0.Data/05_분석데이터/Snd_FE.pkl']